## Домашнее задание

### Задание 1

Для датафрейма **log**  из материалов занятия создайте столбец **source_type** по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится **organic**
- для источников paid и email из России - ставим **ad**
- для источников paid и email не из России - ставим **other**
- все **остальные** варианты берем **из traffic_source без изменений**

In [3]:
# -- Общее для всех -- #

import pandas as pd 

debug = True # Дебаг

In [17]:
log = pd.read_csv('database/visit_log.csv', sep =';')

def classification(data):
    if (data.traffic_source == 'yandex') or (data.traffic_source == 'google') : return 'organic'
    elif (data.traffic_source == 'paid') or (data.traffic_source == 'email') :
        if data.region == 'Russia': return 'ad'
        else: return 'other'
    return data.traffic_source

log['source_type'] = log.apply(classification, axis = 1)
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic


In [18]:
if debug:
    print(log['source_type'].value_counts())
    print()
    print(log['source_type'].value_counts(normalize=True))

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

direct     0.370314
organic    0.285511
ad         0.200127
other      0.144049
Name: source_type, dtype: float64


### Задание 2

В файле **URLs.txt** содержатся url страниц новостного сайта. Вам необходимо **отфильтровать** его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла с датафрейм
- Отфильтруйте страницы с текстом новостей, **используя метод str.contains** и **регулярное выражение** в соответствии с заданным шаблоном

In [78]:
url = pd.read_csv('database/URLs.txt', sep ='\t')
url.head()


,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [80]:
pattern = r'\w+\/\d{8}-([a-zA-Z0-9-]+)'

url.loc[url.url.str.contains(pattern, regex=True), :].head()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [83]:
if debug: #Turn OFF debug if do not want see 'news' column
    url['news'] = url.url.str.extract(pattern, expand=True).fillna('')
url.head()

,url,news
0,/world/,
1,/latest/,
2,/?updated=top,
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...,s-marta-zhizn-rossiyan-suschestvenno-izmenitsy...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...,tramp-pridumal-kak-reshit-ukrainskiy-vopros


### Задание 3

Используйте файл с оценками фильмов **ml-latest-small/ratings.csv**. Посчитайте среднее время жизни пользователей, которые **выставили более 100 оценок**. Под временем жизни понимается **разница между максимальным и минимальным значением столбца timestamp** для данного значения userId.

In [25]:
ratings  = pd.read_csv('database/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [44]:
B = ratings.groupby('userId').count().rating.reset_index() 
a = tuple(B[B.rating >= 100].userId) #кортеж с id пользователей, выставивших более 100 оценок

def timecheck(data):
    return data.timestamp.max() - data.timestamp.min()

#--- Вариант 1. ---#

ratings.loc[ratings.userId.isin(a), : ].groupby('userId').apply(timecheck).mean()

#--- Вариант 2. ---
# C = ratings.loc[ratings.userId.isin(a), : ].groupby('userId').apply(timecheck).reset_index()
# C.columns = ['userId' , 'timedf']
# C.timedf.mean()


39331748.56653992

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо **сформировать две таблицы**:

- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

**Обратите внимание, что в процессе объединения таблиц данные не должны теряться.**

In [69]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'Death Star 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)


In [71]:
#--- таблицу с тремя типами выручки для каждого client_id
rzd_auto_air = rzd.merge(auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
rzd_auto_air.fillna('0')

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093,0,0
1,112,2810,0,0
2,113,10283,57483,0
3,114,5774,83,0
4,115,981,912,81
5,116,0,4834,4
6,117,0,98,13
7,118,0,0,173


In [70]:
#--- аналогичную таблицу по типам выручки с указанием адреса клиента

rzd_auto_air_address = rzd_auto_air.fillna('0').merge(client_base, on='client_id', how='outer')
rzd_auto_air_address

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093,0,0,Комсомольская 4
1,112,2810,0,0,Энтузиастов 8а
2,113,10283,57483,0,Левобережная 1а
3,114,5774,83,0,Мира 14
4,115,981,912,81,Death Star 1
5,116,0,4834,4,Строителей 18
6,117,0,98,13,Панфиловская 33
7,118,0,0,173,Мастеркова 4
